In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os
from sklearn.metrics import classification_report

In [12]:
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [13]:
import os
from torchvision import datasets
from torch.utils.data import DataLoader

data_dir = "/kaggle/input/cards-image-datasetclassification"

train_dataset = datasets.ImageFolder(
    os.path.join(data_dir, "train"),
    transform=train_transforms
)

val_dataset = datasets.ImageFolder(
    os.path.join(data_dir, "valid"),   # usually it's "valid" not "validation"
    transform=val_test_transforms
)

test_dataset = datasets.ImageFolder(
    os.path.join(data_dir, "test"),
    transform=val_test_transforms
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

num_classes = len(train_dataset.classes)
print("Number of Classes:", num_classes)

Number of Classes: 53


In [14]:
import torch
import torch.nn as nn
from torchvision.models import resnet50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load ResNet50 WITHOUT pretrained weights
model = resnet50(weights=None)

# Freeze all layers (optional now since not pretrained)
for param in model.parameters():
    param.requires_grad = False

# Replace final layer
num_classes = 53
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)

print("Model loaded (no pretrained weights)")

Model loaded (no pretrained weights)


In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
epochs = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}] Loss: {running_loss/len(train_loader):.4f} | Accuracy: {train_acc:.2f}%")

In [18]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_acc = 100 * correct / total
print("Validation Accuracy:", val_acc)

Validation Accuracy: 2.641509433962264


In [19]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.numpy())

print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))

                   precision    recall  f1-score   support

     ace of clubs       0.00      0.00      0.00         5
  ace of diamonds       0.00      0.00      0.00         5
    ace of hearts       0.00      0.00      0.00         5
    ace of spades       0.00      0.00      0.00         5
   eight of clubs       0.00      0.00      0.00         5
eight of diamonds       0.00      0.00      0.00         5
  eight of hearts       0.00      0.00      0.00         5
  eight of spades       0.00      0.00      0.00         5
    five of clubs       0.00      0.00      0.00         5
 five of diamonds       0.00      0.00      0.00         5
   five of hearts       0.00      0.00      0.00         5
   five of spades       0.00      0.00      0.00         5
    four of clubs       0.00      0.00      0.00         5
 four of diamonds       0.00      0.00      0.00         5
   four of hearts       0.00      0.00      0.00         5
   four of spades       0.00      0.00      0.00       

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
